# Data Processing

In this notebook, we will prepare the dataset for modeling.  

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the dataset
df = pd.read_excel(r"../data/raw/hanoi_daily.xlsx")

In [3]:
# Xác định loại dữ liệu
numerical_features = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

print("Numeric features:", numerical_features)
print("Categorical features:", categorical_features)

Numeric features: ['tempmax', 'tempmin', 'temp', 'feelslikemax', 'feelslikemin', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob', 'precipcover', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir', 'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation', 'solarenergy', 'uvindex', 'severerisk', 'moonphase']
Categorical features: ['name', 'datetime', 'preciptype', 'sunrise', 'sunset', 'conditions', 'description', 'icon', 'stations']


In [10]:
# Chia train test

In [4]:
# Kiểm tra missing values và duplicates
print("Số lượng giá trị thiếu:\n", df.isnull().sum())
print("\nTỷ lệ thiếu (%):\n", (df.isnull().mean() * 100).round(2))

dupes_daily = df.duplicated().sum()
print(f"\nSố dòng trùng lặp: {dupes_daily}")

Số lượng giá trị thiếu:
 name                   0
datetime               0
tempmax                0
tempmin                0
temp                   0
feelslikemax           0
feelslikemin           0
feelslike              0
dew                    0
humidity               0
precip                 0
precipprob             0
precipcover            0
preciptype          1042
snow                   0
snowdepth              0
windgust               0
windspeed              0
winddir                0
sealevelpressure       0
cloudcover             0
visibility             0
solarradiation         0
solarenergy            0
uvindex                0
severerisk          2566
sunrise                0
sunset                 0
moonphase              0
conditions             0
description            0
icon                   0
stations               0
dtype: int64

Tỷ lệ thiếu (%):
 name                 0.00
datetime             0.00
tempmax              0.00
tempmin              0.00
temp          

In [ ]:
# Xử lý missing values cho từng tập

In [6]:
# Phát hiện outliers bằng phương pháp IQR
numeric_cols_d = df.select_dtypes(include=[np.number]).columns

Q1_d = df[numeric_cols_d].quantile(0.25)
Q3_d = df[numeric_cols_d].quantile(0.75)
IQR_d = Q3_d - Q1_d

outliers_d = ((df[numeric_cols_d] < (Q1_d - 1.5 * IQR_d)) | 
              (df[numeric_cols_d] > (Q3_d + 1.5 * IQR_d))).sum()

print("Số lượng outliers (theo từng cột):\n", outliers_d)

Số lượng outliers (theo từng cột):
 tempmax               3
tempmin               1
temp                  2
feelslikemax          0
feelslikemin         20
feelslike             0
dew                  41
humidity             91
precip              639
precipprob            0
precipcover         169
snow                  0
snowdepth             0
windgust            117
windspeed            59
winddir             664
sealevelpressure      1
cloudcover            0
visibility          119
solarradiation        0
solarenergy           0
uvindex               0
severerisk            0
moonphase             0
dtype: int64


In [ ]:
# Xử lý outliers  cho từng tập

In [13]:
# Ma trận tương quan
corr = df.corr(numeric_only=True)
print(corr)

                   tempmax   tempmin      temp  feelslikemax  feelslikemin  \
tempmax           1.000000  0.909259  0.977722      0.964647      0.889029   
tempmin           0.909259  1.000000  0.969581      0.910259      0.963060   
temp              0.977722  0.969581  1.000000      0.957785      0.942461   
feelslikemax      0.964647  0.910259  0.957785      1.000000      0.899939   
feelslikemin      0.889029  0.963060  0.942461      0.899939      1.000000   
feelslike         0.954692  0.942145  0.974478      0.979062      0.951898   
dew               0.843814  0.932995  0.902911      0.866739      0.872035   
humidity          0.011194  0.226346  0.100473      0.097697      0.140572   
precip            0.056500  0.130930  0.090129      0.099973      0.079126   
precipprob        0.069758  0.251155  0.150019      0.141172      0.186441   
precipcover       0.061395  0.170696  0.106710      0.094612      0.124118   
snow                   NaN       NaN       NaN           NaN    

In [12]:
# Chuẩn hóa (fit cho train rồi transform cho train test)

In [11]:
# Xử lý text (fit cho train rồi transform cho train test)

In [14]:
# Lưu dữ liệu tập train test để training